## Тестовое задание на вакансию data analyst:

Внутри - список работ ученого (**groups**). В каждом списке приведено описание для 1 работы (как правило, это публикация). Для каждой работы может быть несколько описаний.

Необходимо вытащить массив со следующими характеристиками:
1. год издания **publicationDate>year**
2. название работы **title**
3. значения идентификатора типа **doi (workExternalIdentifiers >> значение externalIdentifierId** при условии, что **externalIdentifierType = doi**)
4. значения идентификатора типа **eid (workExternalIdentifiers >> значение externalIdentifierId** при условии, что **externalIdentifierType = eid**)


In [2]:
import requests
import json
import numpy as np
import math

In [16]:
url = 'https://orcid.org/0000-0002-8520-7267/allWorks.json?sort=date&sortAsc=false'

try:
    response = requests.get(url)

    response.raise_for_status()

    data = response.json()

except requests.exceptions.RequestException as e:
    print(f"Ошибка при запросе: {e}")
except ValueError as e:
    print(f"Ошибка при парсинге JSON: {e}")

Для удобства восприятия распечатаем иерархию json файла:

In [17]:
def print_json_hierarchy(data, level=0):
    if isinstance(data, dict):
        for key in data:
            print('  ' * level + f"- {key}")
            print_json_hierarchy(data[key], level + 1)
    elif isinstance(data, list):
        for index, item in enumerate(data):
            print('  ' * level + f"- [{index}]")
            print_json_hierarchy(item, level + 1)


print_json_hierarchy(data)

Выходные данные были обрезаны до нескольких последних строк (5000).
          - month
          - day
          - year
          - timestamp
          - required
          - getRequiredMessage
        - lastModified
          - errors
          - month
          - day
          - year
          - timestamp
          - required
          - getRequiredMessage
        - featuredDisplayIndex
    - featuredDisplayIndex
  - [681]
    - activePutCode
    - defaultPutCode
    - groupId
    - activeVisibility
    - userVersionPresent
    - externalIdentifiers
      - [0]
        - errors
        - externalIdentifierId
          - errors
          - value
          - required
          - getRequiredMessage
        - externalIdentifierType
          - errors
          - value
          - required
          - getRequiredMessage
        - url
        - relationship
          - errors
          - value
          - required
          - getRequiredMessage
        - normalized
          - errors
      

Пример описания работы учёного:

In [5]:
data['groups'][0]

{'activePutCode': 199323490,
 'defaultPutCode': 199323490,
 'groupId': 0,
 'activeVisibility': 'PUBLIC',
 'userVersionPresent': False,
 'externalIdentifiers': [{'errors': [],
   'externalIdentifierId': {'errors': [],
    'value': '10.1109/TCOMM.2025.3642692',
    'required': True,
    'getRequiredMessage': None},
   'externalIdentifierType': {'errors': [],
    'value': 'doi',
    'required': True,
    'getRequiredMessage': None},
   'url': {'errors': [],
    'value': 'https://doi.org/10.1109/TCOMM.2025.3642692',
    'required': True,
    'getRequiredMessage': None},
   'relationship': {'errors': [],
    'value': 'self',
    'required': True,
    'getRequiredMessage': None},
   'normalized': {'errors': [],
    'value': '10.1109/tcomm.2025.3642692',
    'required': False,
    'getRequiredMessage': None},
   'normalizedUrl': {'errors': [],
    'value': 'https://doi.org/10.1109/tcomm.2025.3642692',
    'required': False,
    'getRequiredMessage': None}}],
 'works': [{'visibility': {'errors

Верхнеуровневые атрибуты:

In [6]:
data['groups'][0].keys()

dict_keys(['activePutCode', 'defaultPutCode', 'groupId', 'activeVisibility', 'userVersionPresent', 'externalIdentifiers', 'works', 'featuredDisplayIndex'])

Год издания первой работы:

In [7]:
data['groups'][0]['works'][0]['publicationDate']['year']

'2026'

Название первой работы учёного:

In [8]:
data['groups'][0]['works'][0]['title']['value']

'Deep Unfolding of Iterative Precoding-Based Null-Space Expansion for Multi-User Massive MIMO in Time-Varying Channel'

3. значения идентификатора типа doi (workExternalIdentifiers >> значение externalIdentifierId при условии, что externalIdentifierType = doi)
4. значения идентификатора типа eid (workExternalIdentifiers >> значение externalIdentifierId при условии, что externalIdentifierType = eid)

In [9]:
data['groups'][0]['works'][0]['workExternalIdentifiers']

[{'errors': [],
  'externalIdentifierId': {'errors': [],
   'value': '10.1109/TCOMM.2025.3642692',
   'required': True,
   'getRequiredMessage': None},
  'externalIdentifierType': {'errors': [],
   'value': 'doi',
   'required': True,
   'getRequiredMessage': None},
  'url': {'errors': [],
   'value': 'https://doi.org/10.1109/TCOMM.2025.3642692',
   'required': True,
   'getRequiredMessage': None},
  'relationship': {'errors': [],
   'value': 'self',
   'required': True,
   'getRequiredMessage': None},
  'normalized': {'errors': [],
   'value': '10.1109/tcomm.2025.3642692',
   'required': False,
   'getRequiredMessage': None},
  'normalizedUrl': {'errors': [],
   'value': 'https://doi.org/10.1109/tcomm.2025.3642692',
   'required': False,
   'getRequiredMessage': None}}]

Примеры значений идентификаторов:

In [10]:
data['groups'][0]['works'][0]['workExternalIdentifiers'][0]['externalIdentifierType']['value']

'doi'

In [11]:
data['groups'][0]['works'][0]['workExternalIdentifiers'][0]['externalIdentifierId']['value']

'10.1109/TCOMM.2025.3642692'

In [12]:
data['groups'][125]['works'][0]['workExternalIdentifiers'][0]['externalIdentifierType']['value']

'eid'

In [13]:
data['groups'][125]['works'][0]['workExternalIdentifiers'][0]['externalIdentifierId']['value']

'2-s2.0-85106120185'

Напишем функцию для извлечения необходимой информации, необходимо добиться того, чтобы она не закончила работу с ошибкой при обработке nan, а также не допустила создания дублирующих полностью или частично информацию с описанием работы учёного:

In [14]:
def extract_publication_data(data):
    """
    Извлекает данные о публикациях из заданной структуры данных.

    Параметры:
    data (dict): Структура данных, содержащая информацию о группах и работах.

    Возвращает:
    list: Список словарей, каждый из которых содержит информацию о публикации,
          включая год издания, название работы, DOI и EID.
    """
    results = []

    for group in data.get('groups', []):
        for work in group.get('works', []):
            try:
                year = work.get('publicationDate', {}).get('year', None)
                title = work.get('title', {}).get('value', None)

                doi = None
                eid = None

                descriptions = work.get('workExternalIdentifiers', [])

                for description in descriptions:
                    ext_id_type = description.get('externalIdentifierType', {}).get('value', None)
                    ext_id_value = description.get('externalIdentifierId', {}).get('value', None)

                    if ext_id_type == 'doi':
                        doi = ext_id_value
                    elif ext_id_type == 'eid':
                        eid = ext_id_value

                current_description = {
                    'year': year,
                    'title': title,
                    'doi': doi,
                    'eid': eid
                }

                existing_entry = next((entry for entry in results if entry['title'] == title), None)

                if existing_entry:
                    if (current_description['doi'] is not None and existing_entry['doi'] is None) or (current_description['eid'] is not None and existing_entry['eid'] is None):
                        existing_entry.update(current_description)
                else:
                    results.append(current_description)

            except Exception as e:
                print(f"Ошибка при обработке работы: {e}")

    return results

In [15]:
publications = extract_publication_data(data)
print(json.dumps(publications, indent=4, ensure_ascii=False))

with open('publications.json', 'w', encoding='utf-8') as json_file:
    json.dump(publications, json_file, indent=4, ensure_ascii=False)

print("Данные успешно записаны в файл publications.json")

Ошибка при обработке работы: 'NoneType' object has no attribute 'get'
[
    {
        "year": "2026",
        "title": "Deep Unfolding of Iterative Precoding-Based Null-Space Expansion for Multi-User Massive MIMO in Time-Varying Channel",
        "doi": "10.1109/TCOMM.2025.3642692",
        "eid": null
    },
    {
        "year": "2026",
        "title": "Resource Allocation Schemes for Scalable Panel-Based LIS Surfaces",
        "doi": "10.1109/OJVT.2026.3652908",
        "eid": null
    },
    {
        "year": "2025",
        "title": "On Deep Learning Hybrid Architectures for MIMO-OFDM Channel Estimation",
        "doi": "10.3390/electronics14234692",
        "eid": "2-s2.0-105024537957"
    },
    {
        "year": "2025",
        "title": "Data-Oriented Channel Knowledge Map IoT Transmission Under Hardware Impairments",
        "doi": "10.1109/VTC2025-Spring65109.2025.11174660",
        "eid": "2-s2.0-105019047127"
    },
    {
        "year": "2025",
        "title": "Newmann S